# Results of DDNet Trained on CIFAR10

In this notebook I display results in terms of accuracy on test data for DDNet models trained on the CIFAR10 dataset using various regularization techniques. The DDNet model and the CIFAR10 dataset were set up as in Hoffman 2019 (for all the models). The CIFAR10 data is preprocessed by normalizing using mean [0.5, 0.5, 0.5] and variance [0.5, 0.5, 0.5]. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization with a dropout rate of p_drop = 0.5. I also train a model with Jacobian Regularization without Dropout. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.01, and is reduced to 0.001 and 0.0001 1/3 and 2/3s into training, respectively. The models are trained for 300 epochs (150 000 SGD iterations).

### Imports and Model Loading

In [1]:
import jupyter_black
import numpy as np
import pandas as pd
import pickle
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_CIFAR10
from model_classes import DDNet
from tools import accuracy, ModelInfo

jupyter_black.load()

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_CIFAR10()

# Summary of model
summary_model = DDNet().to(device)
summary(summary_model, (3, 32, 32))

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 30, 30]           1,792
            Conv2d-2           [-1, 64, 28, 28]          36,928
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 12, 12]          73,856
            Conv2d-5          [-1, 128, 10, 10]         147,584
         MaxPool2d-6            [-1, 128, 5, 5]               0
            Linear-7                  [-1, 256]         819,456
           Dropout-8                  [-1, 256]               0
            Linear-9                  [-1, 256]          65,792
          Dropout-10                  [-1, 256]               0
           Linear-11                   [-1, 10]           2,570
Total params: 1,147,978
Trainable params: 1,147,978
Non-trainable params: 0
---------------

In [3]:
# Load models
dataset = "cifar10"

model_names_set = [
    "model_no_reg",
    "model_no_reg_no_dropout",
    "model_l2",
    "model_jacobi",
    "model_jacobi_no_dropout",
    "model_svb",
]
model_names = []

for i in range(5):
    for name in model_names_set:
        model_names.append(f"{name}_{i}")

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [4]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for name in model_names_set:
    accuracies = []
    for i in range(5):  # Assuming 5 versions of each model
        model_name = f"{name}_{i}"
        model = models[model_name].model
        acc = accuracy(model, test_loader, device)
        accuracies.append(acc)

    # Calculate mean accuracy
    mean_accuracy = np.mean(accuracies)

    # Calculate standard error
    std_error = stats.sem(accuracies)

    # Calculate confidence interval
    CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

    # Store results in the data dictionary
    accuracy_data[f"{name}"] = (mean_accuracy, CI)

In [5]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

,Model,Accuracy +/- CI (95%)
0,model_no_reg,80.194 +/- 0.398
1,model_no_reg_no_dropout,77.406 +/- 0.717
2,model_l2,81.418 +/- 0.217
3,model_jacobi,80.068 +/- 0.076
4,model_jacobi_no_dropout,77.584 +/- 0.418
5,model_svb,81.614 +/- 0.315


### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 50 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the models as rows, and the three different zoom levels (0.025, 0.01 and 0.001) as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation. I use the seed 42, and resolution = 250 for the decision boundary plotting function.

#### Isotropic Total Variation

In [4]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_isotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_isotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

100%|██████████| 30/30 [00:00<00:00, 101.40it/s]


In [5]:
print(
    "Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_isotropic)

Isotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                      0.010  \
                                  mean conf_interval         mean   
model_no_reg_0             3230.984223    621.806683  4130.003904   
model_no_reg_no_dropout_0  2073.778286    518.683772  2182.488886   
model_l2_0                 1929.900318    448.005908  2117.719277   
model_jacobi_0              2868.97209    696.649939  4718.000686   
model_jacobi_no_dropout_0  2260.734833    504.361608  2430.579468   
model_svb_0                4198.652889    523.070869   3096.62814   
model_no_reg_1             2900.707299    678.826069  4903.904709   
model_no_reg_no_dropout_1  2014.461493    540.722457  1832.240638   
model_l2_1                 1779.260132    424.753381  1509.425009   
model_jacobi_1             2608.218505    539.764642   2323.30296   
model_jacobi_no_dropout_1  2312.455201    472.458094  2832.980634   
model_svb_1                3338.620141    609.034283    4421.6515   
model_no_reg_2             2659.533851    522.321132  3361.221269   
model_no_reg_no_dropout_2  2446.058814     609.22197   1900.21699   
model_l2_2                 1690.315505    428.504966  1821.650857   
model_jacobi_2             1993.055019    479.024018  2884.736354   
model_jacobi_no_dropout_2  2533.387555    521.128815  3231.948831   
model_svb_2                3594.721503    545.783828  4244.888355   
model_no_reg_3             3578.763002    713.403378  4656.279078   
model_no_reg_no_dropout_3   3036.82407    539.238864  3200.895888   
model_l2_3                 1628.474412    436.380359  1333.960279   
model_jacobi_3             2809.863559    632.791453  3551.129254   
model_jacobi_no_dropout_3  2426.972811    456.043966  2658.311528   
model_svb_3                2637.384655     494.45029  4111.301839   
model_no_reg_4             3145.138466    721.338808  3352.620644   
model_no_reg_no_dropout_4  2664.993021    604.292479  2942.192841   
model_l2_4                 2120.212424    583.056844  2470.330313   
model_jacobi_4             2350.900335    604.122905  2020.214415   
model_jacobi_no_dropout_4  2841.109384    551.594384  3079.291479   
model_svb_4                3382.859393    582.759099  5227.750899   

                                               0.001                
                          conf_interval         mean conf_interval  
model_no_reg_0               722.991808  5341.046575     860.78618  
model_no_reg_no_dropout_0    403.940593  2826.427328    393.873631  
model_l2_0                   302.795923  2496.824314    397.545799  
model_jacobi_0               689.689796  5760.271775    871.311712  
model_jacobi_no_dropout_0    363.984529  2433.740831    450.835178  
model_svb_0                  423.358764   306.081875     46.482188  
model_no_reg_1               704.839352  7424.636717    814.367058  
model_no_reg_no_dropout_1    433.572871  2467.838479    342.277812  
model_l2_1                   299.756499  2832.521532    504.524536  
model_jacobi_1               495.244944   998.270265     316.07948  
model_jacobi_no_dropout_1    464.548189  4466.396244    526.331068  
model_svb_1                  399.168127   659.653904    173.820578  
model_no_reg_2               520.527314  4690.028829    583.798952  
model_no_reg_no_dropout_2    449.043567  2205.924691    418.760326  
model_l2_2                   267.133167  2385.011501    375.272991  
model_jacobi_2               436.622215    3683.4844    547.760856  
model_jacobi_no_dropout_2    505.517361  3530.303834    572.008273  
model_svb_2                  426.564728  4110.336887    519.496761  
model_no_reg_3               680.484648  4833.953546    845.326385  
model_no_reg_no_dropout_3    498.474526  3421.623141    465.322903  
model_l2_3                   353.437916  2291.227467    474.957356  
model_jacobi_3               568.689485  5177.480199     763.47057  
model_jacobi_no_dropout_3    361.183821  2365.468547    420.908867  
model_svb_3                  440.990939   628.523655    166.097244  
model_no_reg_4               4

#### Anisotropic Total Variation

In [6]:
# Load pretrained results

# Define zoom levels
zoom_levels = [0.025, 0.01, 0.001]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=model_names, columns=cols)

# Loop over the selected models
for name in tqdm(model_names):
    # Load the results
    with open(
        f"./total_variation_{dataset}_models/{name}_total_anisotropic_variation.pkl",
        "rb",
    ) as f:
        df_results = pickle.load(f)

    # Copy the results to the dataframe
    for zoom in zoom_levels:
        df_results_anisotropic.loc[name, (zoom, "mean")] = df_results.loc[
            name, (zoom, "mean")
        ]
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = df_results.loc[
            name, (zoom, "conf_interval")
        ]

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:00<00:00, 93.74it/s]


In [7]:
print(
    "Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)"
)
display(df_results_anisotropic)

Anisotropic Total variation for three different zoom levels (mean and 95% CI for 50 generated images at each zoom level)


0.025                  0.010                \
                              mean conf_interval     mean conf_interval   
model_no_reg_0             3528.48    676.649425  4503.68    790.960381   
model_no_reg_no_dropout_0  2282.72    573.978364  2398.48    445.071854   
model_l2_0                 2122.54    491.579579   2330.3    331.838672   
model_jacobi_0             3149.32    763.120291  5210.76    757.205573   
model_jacobi_no_dropout_0  2484.54    557.148753  2693.32    408.656256   
model_svb_0                4595.94    567.920753  3398.28    470.590972   
model_no_reg_1             3177.06    738.482444  5406.44    766.934493   
model_no_reg_no_dropout_1  2222.32    600.080215  2013.64    475.769516   
model_l2_1                  1957.8    471.160115  1657.24    329.664018   
model_jacobi_1             2865.52    593.194924  2556.04    549.812151   
model_jacobi_no_dropout_1  2531.14    513.144299   3102.3    501.675334   
model_svb_1                3646.14    660.528394  4806.16    433.107382   
model_no_reg_2              2916.4    570.392216  3721.76    579.511977   
model_no_reg_no_dropout_2  2683.52    667.572981  2094.52    498.631258   
model_l2_2                  1864.6    472.150111  2015.64    295.758296   
model_jacobi_2             2214.56    534.385113  3200.96    488.676154   
model_jacobi_no_dropout_2  2794.04    579.268226  3570.92    559.488424   
model_svb_2                3960.36    603.701467  4654.16    472.037652   
model_no_reg_3             3909.22    778.687101   5132.3    754.836073   
model_no_reg_no_dropout_3   3351.2    600.657959   3530.2    546.226737   
model_l2_3                 1787.18    476.875273  1472.58    391.203598   
model_jacobi_3             3072.62    689.493839  3896.08     620.74651   
model_jacobi_no_dropout_3  2678.76    506.101703  2919.84    391.572022   
model_svb_3                2876.26    544.713602  4491.44    482.229981   
model_no_reg_4             3455.22    799.906471  3678.46    531.557644   
model_no_reg_no_dropout_4  2910.86    658.971209  3206.62    458.481651   
model_l2_4                 2304.84    634.661373  2705.18    402.642356   
model_jacobi_4             2577.32    660.675444  2217.34    391.162702   
model_jacobi_no_dropout_4   3114.8    602.038411  3394.66    485.500666   
model_svb_4                3717.72     647.45254  5719.96    444.140292   

                             0.001                
                              mean conf_interval  
model_no_reg_0             5838.76    942.190527  
model_no_reg_no_dropout_0  3134.92    452.441204  
model_l2_0                 2742.12    438.998218  
model_jacobi_0             6369.24    948.811034  
model_jacobi_no_dropout_0  2698.46    495.612887  
model_svb_0                 338.64      51.85476  
model_no_reg_1              8175.9    916.136857  
model_no_reg_no_dropout_1  2721.12    375.391712  
model_l2_1                 3076.58    542.075902  
model_jacobi_1              1090.1     344.76404  
model_jacobi_no_dropout_1   4896.8    557.833443  
model_svb_1                 726.84    205.803678  
model_no_reg_2             5214.44    655.343874  
model_no_reg_no_dropout_2  2446.98    452.555095  
model_l2_2                 2623.16    407.285865  
model_jacobi_2              4094.2    608.757818  
model_jacobi_no_dropout_2  3942.12    652.159268  
model_svb_2                4454.48    563.792831  
model_no_reg_3             5331.98    926.716702  
model_no_reg_no_dropout_3  3736.68    503.218422  
model_l2_3                 2542.86    529.210185  
model_jacobi_3             5704.88    830.236218  
model_jacobi_no_dropout_3  2637.96    476.366835  
model_svb_3                 694.88    182.683194  
model_no_reg_4             2623.78    685.320422  
model_no_reg_no_dropout_4  2917.94    536.818595  
model_l2_4                 3562.62    691.287679  
model_jacobi_4              2993.6    310.811958  
model_jacobi_no_dropout_4  2059.66    453.993027  
model_svb_4                1379.36     294.6210